In [ ]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [ ]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


In [ ]:
'''
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
'''

# Search for tweets within the target countries
usa = '96683cc9126741d1'
canada = "3376992a082d67c7"
great_britain = '6416b8512febefc9'
Australia = '3f14ce28dc7c4566'
new_zealand = "0b039ce18ddf1cb9"

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
max_count_4_each_country = 1500

#add usa
#tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ usa + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
#time.sleep(10)

#add Canada
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ canada + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add "Great" Britian
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ great_britain + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add Australia
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ Australia + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
time.sleep(10)

#add new zealand
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ new_zealand + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)


'''
#return this if you are using pages again!!!!!
for page in ALL_pages:
    tweets_list.extend(page)
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]    
'''
print('done')

In [ ]:
tweets_list = [t for t in tweets_list]
len(tweets_list)

In [ ]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
#location = [];

city = [];
country = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
respectability = []
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
hashtags_count = []
trending_for_area = []

users = [];

print('loading users')

arr = []
i = 1

for t in tweets_list:
    tweet += t.full_text;
    
    users += t.user;
    
    word_count += countWordsInString(t.full_text))
    
    likes += t.favorite_count
    
    retweet_count += t.retweet_count
    
    is_quote_status += t.is_quote_status
    
    date += t.created_at.isoformat()[:10]
    
    tweet_id += t.id
    
    ht = [t['text'] for t in t.entities['hashtags']]
    ht_count = len(ht)
    
    if ht == []:
        ht = np.nan
        ht_count = 0
    
    hashtags += ht
    hashtags_count = ht_count
    
    
    city += t.place.name
    country += t.place.country    
    
    trending_for_area += getTrends(t.place.name,t.place.country)
        
for x in users:
    if i%26 == 0:
        print("waiting")
        time.sleep(6)
    j = x._json
    
    #opens the page
    DRIVER.get('https://twitter.com/' + screen_name) 
    screen_name = j['screen_name']    
    
    #get the easy stuff
    user_name += screen_name
    following += j['friends_count']
    followers += j['followers_count']
    
    respectability += float(j['friends_count']) / float(j['followers_count'])
    joined += x.created_at.isoformat()[:10]
    tweet_count_user += x.statuses_count
    verified += x.verified
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age += np.nan
    else:
        age += tmp
    
    
    gender += getGender(j['description'],j['location'],j['name'])
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

In [ ]:
max = len(users)

In [ ]:
dick = {"name":user_name[:max],
        "age":age[:max],
        'city':city[:max],
        'country':country[:max],
        "gender":gender[:max],
        'join_date':joined[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'respectability':respectability[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'word count': word_count[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'hashtag count':hashtags_count[:max],
        'trending': trending_for_area[:max],
        'date':date[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [ ]:
df

In [ ]:
DRIVER.quit()

In [ ]:
df.to_csv('7_5k.csv')